In [71]:
import os
import pandas as pd
import glob
import nbformat

In [72]:
directory = 'C:/Users/FANNY MARIA D SOUZA/Capstone_Movement/Movement_CompletedSteps_ComplexPie/'
file_pattern = directory + '*_Movement.ipynb'

#### Extracting Position Features

In [73]:
def extract_features_summary(file_path):
    with open(file_path, 'r') as file:
        nb = nbformat.read(file, as_version=4)
    
    local_context = {}
    for cell in nb.cells:
        if cell.cell_type == 'code':
            try:
                # Check if the cell contains plotting code and skip it
                if 'plt.show()' in cell.source or 'plt.plot' in cell.source or 'plt.bar' in cell.source:
                    continue
                exec(cell.source, {}, local_context)
                if 'total_distances_df' in local_context:
                    return local_context['total_distances_df']
            except Exception as e:
                print(f"Error executing cell in {file_path}: {e}")
    return None

total_distance = []

# Extract data from each notebook
for file_path in glob.glob(file_pattern):
    participant_id = os.path.basename(file_path)[:4] 
    df = extract_features_summary(file_path)
    if df is not None:
        df['Participant'] = participant_id 
        total_distance.append(df)
    else:
        print(f"No features_summary found in {file_path}")

combined_features_summary = pd.concat(total_distance, ignore_index=True)
combined_features_summary

,Tracker,Total Distance,Participant
0,Chest,3395.496127,C001
1,LeftShoulder,4723.892731,C001
2,RightShoulder,0.000000,C001
3,LeftElbow,4943.053122,C001
4,RightElbow,0.000000,C001
...,...,...,...
123,LeftElbow,3136.425590,C032
124,RightElbow,11331.897470,C032
125,MotionControllerLeft,4364.854730,C032
126,MotionControllerRight,16067.186816,C032


In [74]:
pivoted_df = combined_features_summary.pivot_table(
    index='Participant', 
    columns='Tracker', 
    values='Total Distance'
).reset_index()

pivoted_df


Tracker,Participant,Chest,HeadCamera,LeftElbow,LeftShoulder,MotionControllerLeft,MotionControllerRight,RightElbow,RightShoulder
0,C001,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,0.000000,0.000000
1,C003,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680
2,C006,0.000000,13718.031017,0.000000,0.000000,11931.234830,13994.063894,0.000000,0.000000
3,C008,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385
4,C009,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661
5,C012,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823
6,C013,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950
7,C016,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522
8,C017,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095
9,C020,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790


In [75]:
from sklearn.impute import KNNImputer
import numpy as np

imputer = KNNImputer(n_neighbors=1)

numerical_cols = pivoted_df.select_dtypes(include=['float64', 'int64']).columns
pivoted_df[numerical_cols] = pivoted_df[numerical_cols].replace(0, np.nan)

pivoted_df[numerical_cols] = imputer.fit_transform(pivoted_df[numerical_cols])
pivoted_df

Tracker,Participant,Chest,HeadCamera,LeftElbow,LeftShoulder,MotionControllerLeft,MotionControllerRight,RightElbow,RightShoulder
0,C001,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587
1,C003,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680
2,C006,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790
3,C008,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385
4,C009,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661
5,C012,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823
6,C013,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950
7,C016,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522
8,C017,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095
9,C020,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790


In [76]:
new_column_names = {
    'Chest': 'Chest Total Distance',
    'HeadCamera': 'HeadCamera Total Distance',
    'LeftElbow': 'LeftElbow Total Distance',
    'LeftShoulder': 'LeftShoulder Total Distance',
    'MotionControllerLeft': 'MotionControllerLeft Total Distance',
    'MotionControllerRight': 'MotionControllerRight Total Distance',
    'RightElbow': 'RightElbow Total Distance',
    'RightShoulder': 'RightShoulder Total Distance'
}

# Rename columns
pivoted_df = pivoted_df.rename(columns=new_column_names)
pivoted_df

Tracker,Participant,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance
0,C001,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587
1,C003,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680
2,C006,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790
3,C008,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385
4,C009,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661
5,C012,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823
6,C013,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950
7,C016,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522
8,C017,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095
9,C020,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790


In [77]:
pivoted_df.to_csv('PositionDistance_ComplexPie.csv', index=False)

In [78]:
bscore=pd.read_csv('M_BDScore.csv')

In [79]:
final_df = pd.merge(bscore, pivoted_df, on='Participant', how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790


In [80]:
addapples=pd.read_csv('ComplexPie_AddApples.csv')

In [81]:
addapples.columns

Index(['Participant', 'Block Design Raw Score', 'Step',
       'Chest Distance - Add Apples', 'HeadCamera Distance - Add Apples',
       'LeftElbow Distance - Add Apples', 'LeftShoulder Distance - Add Apples',
       'MotionControllerLeft Distance - Add Apples',
       'MotionControllerRight Distance - Add Apples',
       'RightElbow Distance - Add Apples',
       'RightShoulder Distance - Add Apples'],
      dtype='object')

In [82]:
if 'Block Design Raw Score' in addapples.columns:
    addapples['Block Design Raw Score'] = addapples['Block Design Raw Score'].astype(int)
addapples = addapples.drop(columns=['Step'])

final_df = pd.merge(final_df, addapples, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,Chest Distance - Add Apples,HeadCamera Distance - Add Apples,LeftElbow Distance - Add Apples,LeftShoulder Distance - Add Apples,MotionControllerLeft Distance - Add Apples,MotionControllerRight Distance - Add Apples,RightElbow Distance - Add Apples,RightShoulder Distance - Add Apples
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587,604.934512,891.537804,751.720262,712.561806,979.341382,3126.059197,1780.217261,1068.079809
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680,536.236211,735.326429,2176.457761,648.630988,1107.648439,3382.430048,1766.212367,1244.350696
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790,1159.659473,3379.233117,2539.534217,2253.539045,2501.928454,4404.150571,2765.896671,2106.291680
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385,2304.163176,2880.191997,3410.427345,3041.911670,3421.640744,8070.078953,6199.743605,4392.008600
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661,2344.988669,3274.813739,2672.153843,3108.914340,2660.870183,8043.298615,5350.501010,3900.905669
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823,722.269516,876.952733,1281.925462,994.361602,1805.556497,3998.861141,2327.116645,1475.884196
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950,2371.114133,2803.826064,3753.108766,3354.837484,4728.598911,5798.915955,4207.281884,3285.004349
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522,1159.659473,1743.678861,2539.534217,2253.539045,3095.585149,3733.026661,2765.896671,2106.291680
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095,192.146733,435.118368,627.883894,430.763953,784.540840,1637.282136,1009.332362,1458.600894
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790,469.184298,683.903486,790.653252,602.483685,1058.445109,1975.633517,1238.733965,826.424108


In [83]:
addbutter=pd.read_csv('ComplexPie_AddButter.csv')

In [84]:
if 'Block Design Raw Score' in addbutter.columns:
    addapples['Block Design Raw Score'] = addbutter['Block Design Raw Score'].astype(int)
addbutter = addbutter.drop(columns=['Step'])

final_df = pd.merge(final_df, addbutter, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Apples,RightShoulder Distance - Add Apples,Chest Distance - Add Butter,HeadCamera Distance - Add Butter,LeftElbow Distance - Add Butter,LeftShoulder Distance - Add Butter,MotionControllerLeft Distance - Add Butter,MotionControllerRight Distance - Add Butter,RightElbow Distance - Add Butter,RightShoulder Distance - Add Butter
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587,...,1780.217261,1068.079809,571.126297,849.580165,931.140785,945.605559,1027.631565,1652.806324,972.735166,880.242172
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680,...,1766.212367,1244.350696,138.907986,221.940560,358.531351,277.606829,461.167238,310.717666,214.293608,176.908457
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790,...,2765.896671,2106.291680,138.907986,219.353317,358.531351,277.606829,480.163224,275.516879,214.293608,176.908457
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385,...,6199.743605,4392.008600,486.340683,710.046255,545.433451,590.988574,587.834617,1608.015947,1291.257148,984.417755
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661,...,5350.501010,3900.905669,427.487776,540.538002,486.192224,477.866274,476.424223,1306.985480,963.798138,706.778471
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823,...,2327.116645,1475.884196,219.754060,277.790749,462.263152,362.507467,559.533869,718.926158,541.175050,359.358999
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950,...,4207.281884,3285.004349,267.877842,351.338373,390.964080,332.355571,502.944251,757.758025,542.357526,407.948128
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522,...,2765.896671,2106.291680,441.033194,793.549599,935.947123,894.158513,1241.037214,1095.528796,972.735166,880.242172
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095,...,1009.332362,1458.600894,232.424691,486.651381,324.683484,308.230312,421.582352,1018.188932,802.982397,558.415926
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790,...,1238.733965,826.424108,232.986416,456.555122,719.507307,567.454135,937.586417,478.588233,525.960772,292.956383


In [85]:
addcinnamon=pd.read_csv('ComplexPie_AddCinnamon.csv')

In [86]:
if 'Block Design Raw Score' in addcinnamon.columns:
    addcinnamon['Block Design Raw Score'] = addcinnamon['Block Design Raw Score'].astype(int)
addcinnamon = addcinnamon.drop(columns=['Step'])

final_df = pd.merge(final_df, addcinnamon, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Butter,RightShoulder Distance - Add Butter,Chest Distance - Add Cinnamon,HeadCamera Distance - Add Cinnamon,LeftElbow Distance - Add Cinnamon,LeftShoulder Distance - Add Cinnamon,MotionControllerLeft Distance - Add Cinnamon,MotionControllerRight Distance - Add Cinnamon,RightElbow Distance - Add Cinnamon,RightShoulder Distance - Add Cinnamon
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587,...,972.735166,880.242172,332.507892,473.722655,552.750738,459.671170,679.054074,936.655177,742.580899,627.220704
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680,...,214.293608,176.908457,63.216986,94.901605,87.897351,50.392992,102.029885,386.250360,232.732397,151.361513
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790,...,214.293608,176.908457,471.875852,467.617148,547.941439,515.647652,845.146150,791.188534,742.580899,627.220704
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385,...,1291.257148,984.417755,237.613261,348.966293,357.126817,368.571224,301.645953,850.896419,749.151677,546.631920
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661,...,963.798138,706.778471,113.008018,181.765418,155.237545,152.036453,144.311283,381.119681,328.953711,222.022422
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823,...,541.175050,359.358999,206.194707,239.530605,267.236872,221.274184,218.697388,806.863162,560.781051,358.166667
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950,...,542.357526,407.948128,471.875852,538.685216,547.941439,515.647652,672.623875,851.695093,742.580899,627.220704
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522,...,972.735166,880.242172,154.036838,322.836796,272.788526,238.180157,316.876245,722.745521,520.328291,359.905787
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095,...,802.982397,558.415926,60.180033,188.159975,306.440016,155.317077,219.915122,468.420989,382.154188,245.550354
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790,...,525.960772,292.956383,712.336456,1063.913755,1363.010567,1054.180742,2124.107917,3096.883017,2017.308953,1222.728386


In [87]:
addcrusttop=pd.read_csv('ComplexPie_AddCrustTop.csv')

In [88]:
if 'Block Design Raw Score' in addcrusttop.columns:
    addcrusttop['Block Design Raw Score'] = addcrusttop['Block Design Raw Score'].astype(int)
addcrusttop = addcrusttop.drop(columns=['Step'])

final_df = pd.merge(final_df, addcrusttop, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Cinnamon,RightShoulder Distance - Add Cinnamon,Chest Distance - Add Crusttop,HeadCamera Distance - Add Crusttop,LeftElbow Distance - Add Crusttop,LeftShoulder Distance - Add Crusttop,MotionControllerLeft Distance - Add Crusttop,MotionControllerRight Distance - Add Crusttop,RightElbow Distance - Add Crusttop,RightShoulder Distance - Add Crusttop
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587,...,742.580899,627.220704,209.483986,263.142734,294.557339,274.961739,290.218322,618.742198,500.926769,351.598562
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680,...,232.732397,151.361513,76.421864,90.498375,49.316725,84.409906,71.150257,424.487148,305.036468,189.421069
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790,...,742.580899,627.220704,201.770038,298.058736,366.606081,304.984571,431.436902,333.414390,483.479056,362.584271
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385,...,749.151677,546.631920,309.416528,397.550724,493.328771,445.495948,501.744381,713.207780,643.766223,500.766993
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661,...,328.953711,222.022422,201.770038,304.271309,366.606081,304.984571,383.776629,603.685648,483.479056,362.584271
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823,...,560.781051,358.166667,295.475928,313.922341,774.968875,520.193362,824.403049,751.640742,641.536370,440.378118
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950,...,742.580899,627.220704,339.446726,380.699478,372.494542,383.665401,448.364317,642.754633,529.379843,432.265120
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522,...,520.328291,359.905787,359.238444,667.215096,870.808695,775.141509,1063.078550,1119.697352,967.942231,700.132171
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095,...,382.154188,245.550354,198.353102,289.772655,116.734822,243.133306,269.718865,690.086769,572.059254,386.049984
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790,...,2017.308953,1222.728386,193.082118,305.221242,304.799172,272.935211,354.539324,641.650769,500.926769,351.598562


In [89]:
addsugar=pd.read_csv('ComplexPie_AddSugar.csv')

In [90]:
if 'Block Design Raw Score' in addsugar.columns:
    addsugar['Block Design Raw Score'] = addsugar['Block Design Raw Score'].astype(int)
addsugar = addsugar.drop(columns=['Step'])

final_df = pd.merge(final_df, addsugar, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Crusttop,RightShoulder Distance - Add Crusttop,Chest Distance - Add Sugar,HeadCamera Distance - Add Sugar,LeftElbow Distance - Add Sugar,LeftShoulder Distance - Add Sugar,MotionControllerLeft Distance - Add Sugar,MotionControllerRight Distance - Add Sugar,RightElbow Distance - Add Sugar,RightShoulder Distance - Add Sugar
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587,...,500.926769,351.598562,502.801470,633.893035,683.326804,670.784469,785.199803,1612.699496,973.918163,754.679255
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680,...,305.036468,189.421069,418.830766,537.979561,616.722995,622.906000,1034.522639,1007.308532,825.225418,662.538180
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790,...,483.479056,362.584271,496.728131,401.715093,616.093445,469.303269,530.229085,538.692684,643.145783,541.571508
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385,...,643.766223,500.766993,513.335344,552.460818,675.934898,773.141680,674.286850,1209.754735,1046.956623,821.064589
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661,...,483.479056,362.584271,1170.091011,1528.348308,1472.956809,1469.479488,1482.519391,2502.618246,2211.283018,1746.598537
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823,...,641.536370,440.378118,496.728131,329.120375,616.093445,469.303269,611.864382,893.608139,643.145783,541.571508
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950,...,529.379843,432.265120,533.901216,687.670072,1109.740023,854.268322,1319.687548,665.341140,552.266341,566.429485
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522,...,967.942231,700.132171,1613.431626,1376.832093,1399.272791,1344.856056,1664.596131,2145.661525,1743.952683,1356.891678
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095,...,572.059254,386.049984,673.565419,897.502506,1280.198513,1021.653608,1562.186440,1850.276821,1324.604149,949.789831
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790,...,500.926769,351.598562,837.372538,2750.808438,1727.712419,1625.934333,1265.401897,1595.414344,2258.026615,1383.387934


In [91]:
addwater=pd.read_csv('ComplexPie_AddWater.csv')

In [92]:
if 'Block Design Raw Score' in addwater.columns:
    addwater['Block Design Raw Score'] = addwater['Block Design Raw Score'].astype(int)
addwater = addwater.drop(columns=['Step'])

final_df = pd.merge(final_df, addwater, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Sugar,RightShoulder Distance - Add Sugar,Chest Distance - Add Water,HeadCamera Distance - Add Water,LeftElbow Distance - Add Water,LeftShoulder Distance - Add Water,MotionControllerLeft Distance - Add Water,MotionControllerRight Distance - Add Water,RightElbow Distance - Add Water,RightShoulder Distance - Add Water
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587,...,973.918163,754.679255,100.789411,148.119862,123.624984,133.256456,147.716461,329.876648,218.324367,184.970963
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680,...,825.225418,662.538180,73.014737,81.427021,145.737973,114.244999,212.487411,237.536156,160.975607,109.222516
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790,...,643.145783,541.571508,606.043573,7416.562265,681.359516,673.056011,5474.310174,4194.656740,764.591321,706.371067
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385,...,1046.956623,821.064589,182.744824,265.491068,199.151451,215.109480,170.052774,445.012205,423.942647,352.392494
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661,...,2211.283018,1746.598537,153.981014,232.855560,174.022086,183.790053,164.283716,406.946399,372.477972,259.926184
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823,...,643.145783,541.571508,175.354869,196.012047,297.680835,225.141570,401.264098,323.611320,249.191077,209.827920
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950,...,552.266341,566.429485,170.883674,209.341897,394.663267,292.824260,557.470761,367.739503,292.078650,243.900349
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522,...,1743.952683,1356.891678,316.064957,286.643499,281.679164,288.788837,346.585941,559.016239,502.062630,352.823481
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095,...,1324.604149,949.789831,46.595893,198.461201,62.507003,95.368106,137.648673,318.221513,244.712189,181.892297
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790,...,2258.026615,1383.387934,110.717918,214.412853,290.653925,227.620506,341.882591,305.704861,422.832032,169.617637


In [93]:
addpie=pd.read_csv('ComplexPie_Putpieinoven.csv')

In [94]:
if 'Block Design Raw Score' in addpie.columns:
    addpie['Block Design Raw Score'] = addpie['Block Design Raw Score'].astype(int)
addpie = addpie.drop(columns=['Step'])

final_df = pd.merge(final_df, addpie, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Water,RightShoulder Distance - Add Water,Chest Distance - Add Pie,HeadCamera Distance - Add Pie,LeftElbow Distance - Add Pie,LeftShoulder Distance - Add Pie,MotionControllerLeft Distance - Add Pie,MotionControllerRight Distance - Add Pie,RightElbow Distance - Add Pie,RightShoulder Distance - Add Pie
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587,...,218.324367,184.970963,417.705357,646.987973,521.009879,508.300452,499.222944,1049.358871,864.452598,701.028237
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680,...,160.975607,109.222516,281.770756,556.204339,681.118664,554.932002,732.631683,1029.531151,733.825350,632.000105
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790,...,764.591321,706.371067,422.700850,531.163977,511.312859,504.001243,435.572246,1020.632468,864.452598,701.028237
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385,...,423.942647,352.392494,603.364727,736.761957,872.864373,780.591941,913.534534,1202.199385,1025.486016,813.726620
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661,...,372.477972,259.926184,422.700850,583.094933,511.312859,504.001243,484.500992,991.293627,864.452598,701.028237
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823,...,249.191077,209.827920,356.439951,520.127953,723.208582,1187.815747,726.640857,999.174014,842.490863,653.529576
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950,...,292.078650,243.900349,499.460239,581.057959,800.624237,696.579945,940.653910,705.820080,553.620321,511.271771
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522,...,502.062630,352.823481,349.899765,536.055596,553.257403,558.230081,612.744848,686.498503,583.915187,531.949303
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095,...,244.712189,181.892297,439.842838,746.275844,605.319388,620.194841,589.137933,1236.095334,1052.772652,828.146880
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790,...,422.832032,169.617637,451.628664,683.672269,675.585022,605.370138,812.254054,1045.051381,912.612955,739.177843


In [95]:
poursyrup=pd.read_csv('ComplexPie_PourSyrup.csv')

In [96]:
if 'Block Design Raw Score' in poursyrup.columns:
    poursyrup['Block Design Raw Score'] = poursyrup['Block Design Raw Score'].astype(int)
poursyrup = poursyrup.drop(columns=['Step'])

final_df = pd.merge(final_df, poursyrup, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Pie,RightShoulder Distance - Add Pie,Chest Distance - Add Syrup,HeadCamera Distance - Add Syrup,LeftElbow Distance - Add Syrup,LeftShoulder Distance - Add Syrup,MotionControllerLeft Distance - Add Syrup,MotionControllerRight Distance - Add Syrup,RightElbow Distance - Add Syrup,RightShoulder Distance - Add Syrup
0,C001,36,3395.496127,4873.394828,4943.053122,4723.892731,5571.116724,11350.784753,8129.374727,6489.176587,...,864.452598,701.028237,131.559940,219.383012,243.410276,230.412376,263.878853,310.688388,212.242779,172.146797
1,C003,58,2441.226143,3706.550475,5360.292055,3692.937327,5354.438631,9325.253301,5944.739917,4504.208680,...,733.825350,632.000105,305.505264,479.528576,429.275107,315.347577,553.793043,524.247051,377.615177,338.691752
2,C006,42,4712.449120,13718.031017,8528.845238,7375.464025,11931.234830,13994.063894,11725.719015,8014.142790,...,864.452598,701.028237,109.338433,132.312614,183.368921,185.855039,150.402195,226.747147,223.311778,199.858685
3,C008,34,5700.541398,7249.311815,8098.794354,7629.508538,8208.691009,17023.989966,13714.711774,10190.067385,...,1025.486016,813.726620,195.377408,263.319124,256.843596,253.717914,290.983899,399.257787,330.720404,316.063602
4,C009,24,6033.585268,8253.252412,7296.033427,7572.354870,7315.455306,17737.162340,13271.209674,9907.909661,...,864.452598,701.028237,94.413539,163.770799,166.705077,154.378053,192.135676,375.831706,228.019399,174.918945
5,C012,54,4288.495977,4712.594053,6729.454648,6143.365294,7840.153705,14035.889455,9658.841557,6739.413823,...,842.490863,653.529576,118.625608,152.790924,164.978799,131.646900,142.912458,397.134531,320.450419,210.551063
6,C013,48,6519.554198,7920.065874,10220.217748,8874.677785,12700.446525,13452.025244,10296.428585,8498.068950,...,553.620321,511.271771,216.891218,265.620868,293.155047,266.067193,319.389614,464.979191,395.509451,321.385721
7,C016,39,5183.757477,7257.163993,8069.332124,7611.329034,9755.215507,13052.393533,10385.884082,8028.321522,...,583.915187,531.949303,98.417169,201.276142,178.119322,150.699296,235.170484,318.691626,212.242779,172.146797
8,C017,36,2594.463613,5302.547139,4429.981578,4482.371709,6464.035372,12528.308020,8977.525132,7180.262095,...,1052.772652,828.146880,118.625608,184.569689,164.978799,97.655706,145.496436,333.731828,276.808112,213.184792
9,C020,30,4712.449120,9261.042902,8528.845238,7375.464025,9747.469647,13801.274714,11725.719015,8014.142790,...,912.612955,739.177843,109.345891,182.016561,158.765160,126.914372,310.721249,388.566801,266.477279,198.197128


In [97]:
final_df.to_csv('Features_ComplexPie_Movement.csv',index=False)